In [11]:
import base64
import openai

from dotenv import load_dotenv
from glob import glob
from tqdm import tqdm
from openai import OpenAI


load_dotenv('../.env')

client = OpenAI()

In [12]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [13]:
PROMPT = """
Generate a description of the painting provided to you. The description is going to be used
to help users find paintings matching how they are feeling and mood. It could also be search against
using features of the painting like the color, the mood, the style, objects. The description will be
vectorized and then used to search against the database of paintings.
"""

In [31]:
image_descriptions = []

for image_path in tqdm(glob('../data/images/*.jpg')):
    image_id = int(image_path.split('/')[-1].split('.')[0])
    if image_id > 20:
        pass
    try:
        encoded_image = encode_image(image_path)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": PROMPT,
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{encoded_image}",
                                "detail": "low"
                            },
                        },
                    ],
                }
            ]
        )
    except openai.APIError as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        pass
    except Exception as e:
        #Handle other exceptions here, e.g. log
        print(f"An error occurred: {e}")
        pass

    image_description = response.choices[0].message
    image_descriptions.append({"image_id": image_id, "image_description": image_description})

  

100%|██████████| 119/119 [16:06<00:00,  8.12s/it] 


In [ ]:
result = []
for obj in image_descriptions:
    result.append((obj['image_id'], obj['image_description'].content))

In [44]:
result

[(63,
  "This painting captures a serene yet dramatic landscape featuring a group of bison near a tranquil water source. Set under a brooding sky filled with expansive clouds, the scene conveys a sense of stillness amidst an impending storm. The foreground is lush with vibrant green vegetation and hints of wildflowers, while majestic trees frame the scene, adding depth and texture. The interplay of light and shadow creates a moody atmosphere, evoking feelings of nostalgia and connection to nature. The overall palette blends earthy tones with shades of blue and gray, enhancing the painting’s emotional weight. Ideal for those seeking a sense of calm in the face of nature's grandeur or reflecting on the beauty of untamed landscapes."),
 (77,
  "This painting presents a serene landscape featuring a calm body of water that gently reflects the soft hues of the sky above. The overall color palette is dominated by soothing shades of lavender and muted greens, evoking a sense of tranquility and

In [46]:
result = []
for obj in tqdm(image_descriptions):
    response = client.embeddings.create(
        input=obj['image_description'].content,
        model="text-embedding-3-small"
    )
    result.append({'id': obj['image_id'], 'values': response.data[0].embedding})

100%|██████████| 119/119 [00:58<00:00,  2.05it/s]


In [51]:
import ndjson
with open('./embeddings.ndjson', 'w') as f:
    ndjson.dump(result, f)

In [57]:
new_result = []
for obj in result:
    new_result.append({'id': str(obj['id']), 'values': obj['embedding']})

In [58]:
len(new_result[0]['values'])

1536

In [59]:
import ndjson
with open('./embedding_new.ndjson', 'w') as f:
    ndjson.dump(new_result, f)